# Demo de modelo BERT para detección de entidades

En este notebook:
- Mostramos como cargar nuestros modelos usando HuggingFace
- Transformamos su output a etiquetas de entidades interpretables

Su objetivo es ejecutar los algoritmos desde cero teniendo sólamente un ambiente conda. Su público objetivo es alguien que se interese en como los modelos tratan los datos. Si el objetivo es únicamente probar el modelo en un input de manera directa ver el notebook **demo_minimalista.ipynb** (que sin embargo requiere instalación previa). Notar que este no fue el mismo ambiente con el cual desarrollamos el proyecto, i.e., este notebook requiere unicamente un ambiente python sin ninguna dependencia instalada previamente.

Fue probado en una máquina linux con ambiente conda instalado, versión de python 3.9.0. 

Instalación de bibliotecas

In [1]:
# instalación de bibliotecas necesarias
!pip install numpy==1.20.0
!pip install pandas==1.4.3
!pip install scipy==1.9.1

  Using cached pandas-1.4.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.3
    Uninstalling pandas-1.2.3:
      Successfully uninstalled pandas-1.2.3
  Using cached scipy-1.9.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.9 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.9.3
    Uninstalling scipy-1.9.3:
      Successfully uninstalled scipy-1.9.3


In [2]:
!pip3 install torch
!pip install transformers==4.23.1

     |████████████████████████████████| 887.4 MB 40 kB/s  eta 0:00:0116
     |████████████████████████████████| 21.0 MB 30.5 MB/s eta 0:00:01
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
     |████████████████████████████████| 849 kB 15.0 MB/s eta 0:00:01
     |████████████████████████████████| 557.1 MB 34 kB/s  eta 0:00:012
     |████████████████████████████████| 317.1 MB 92 kB/s s eta 0:00:01     |█████████████▊                  | 136.4 MB 382 kB/s eta 0:07:53
  Using cached transformers-4.23.1-py3-none-any.whl (5.3 MB)
  Using cached filelock-3.8.2-py3-none-any.whl (10 kB)
  Using cached regex-2022.10.31-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (769 kB)
     |████████████████████████████████| 7.6 MB 253 kB/s eta 0:00:011
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached huggingface_hub-0.11.1-py3-no

Descarga de modelos

In [3]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

MODEL = 'ccarvajal/beto-prescripciones-medicas'

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForTokenClassification.from_pretrained(MODEL, num_labels=11, ignore_mismatched_sizes=True)

Downloading:   0%|          | 0.00/600 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/735k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/437M [00:00<?, ?B/s]

Para ambos tokenizer y modelos, se puede guardar una copia local usando el método .save_pretrained()

### Output del modelo

Definimos dos funciones auxiliares que nos ayudarán con el etiquetado

In [4]:
from scipy.special import softmax
import numpy as np

def word_ids_method(text,tokenizer):
    """Método que asigna el primer token (subword) de una palabra como representante
        La etiqueta de este token será la etiqueta de la palabra
    Método integrado en tokenize_and_align_labels
    Fuente: https://huggingface.co/docs/transformers/tasks/token_classification

    Argumentos

        text: str o List[str]
            texto a tokenizar, si 
    """
    if not isinstance(text,list):
        text = text.split()
    n_words = len(text)

    tokenized_inputs = tokenizer([text], truncation=True, is_split_into_words=True)
    mask = []
    # for i in range(n_words):
    word_ids = tokenized_inputs.word_ids(batch_index=0)
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            mask.append(0)
        elif word_idx != previous_word_idx:  # Only label the first token of a given word.
            mask.append(1)
        else:
            mask.append(0)
        previous_word_idx = word_idx
    
    return mask

def eval_text(text,tokenizer,model):
    """
    Toma un texto (lista de palabras o string), un tokenizador y modelo de HuggingFace
    Retorna el output del modelo (ids de entidades)
    """
    mask = word_ids_method(text,tokenizer)
    encoded_input = tokenizer(text,return_tensors='pt',is_split_into_words=isinstance(text,list))
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    result = np.argmax(scores,axis=1)

    return result[mask==np.array(1)]

In [5]:
text = "PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias"

print(text)
eval_text(text,tokenizer,model)

PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias


array([ 1,  3,  4,  4,  5,  6,  6,  7,  8,  8,  9, 10, 10])

Tenemos ya un output de nuestro modelo con cinco entidades, sin embargo nos gustaría tener las entidades con sus respectivas etiquetas

In [6]:
ner_dict = {'O': 0,
            'B-ACTIVE_PRINCIPLE': 1,
            'I-ACTIVE_PRINCIPLE': 2,
            'B-FORMA_FARMA':3,
            'I-FORMA_FARMA':4,
            'B-ADMIN': 5,
            'I-ADMIN': 6,
            'B-PERIODICITY': 7,
            'I-PERIODICITY': 8,
            'B-DURATION': 9,
            'I-DURATION': 10
            }

In [7]:
def map_entities(y_pred,map_dict,return_type='list'):
    inv_map = {v: k for k, v in map_dict.items()}
    if return_type == 'list':
        return [inv_map[y] for y in y_pred]
    else:
        return np.array([inv_map[y] for y in y_pred])

In [8]:
entidades = map_entities(eval_text(text,tokenizer,model),ner_dict)
for i, ent in enumerate(entidades):
    print("{}\t{}".format(text.split()[i],ent))

PARACETAMOL	B-ACTIVE_PRINCIPLE
500	B-FORMA_FARMA
MG	I-FORMA_FARMA
COMPRIMIDO	I-FORMA_FARMA
1	B-ADMIN
COMPRIMIDO	I-ADMIN
ORAL	I-ADMIN
cada	B-PERIODICITY
6	I-PERIODICITY
horas	I-PERIODICITY
durante	B-DURATION
3	I-DURATION
dias	I-DURATION


---

## Modelo para etiquetas ADMIN más finas

In [9]:
MODEL_admin = 'ccarvajal/beto-prescripciones-medicas-ADMIN'

tokenizer_admin = AutoTokenizer.from_pretrained(MODEL_admin)
model_admin = AutoModelForTokenClassification.from_pretrained(MODEL_admin, num_labels=7, ignore_mismatched_sizes=True)

Downloading:   0%|          | 0.00/600 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/735k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/437M [00:00<?, ?B/s]

In [10]:
admin_ner_dict = {
    'O': 0,
    'B-CANT': 1,
    'I-CANT': 2,
    'B-UND':3,
    'I-UND':4,
    'B-VIA_ADMIN': 5,
    'I-VIA_ADMIN': 6
}

In [11]:
def get_admin(entidades:list,texto:str) -> str:
    """Retorna un substring correspondiente a aquellas entidades etiquetadas con admin y los indices donde esto ocurre"""
    indices = [i for i, ent in enumerate(entidades) if ent == 'B-ADMIN' or ent == 'I-ADMIN']
    return ' '.join([token for i, token in enumerate(texto.split(' ')) if i in indices]), indices

In [12]:
def etiquetar(texto):
    entidades = map_entities(eval_text(texto,tokenizer,model),ner_dict)
    admin_text, indices = get_admin(entidades,texto)
    entidades_admin = map_entities(eval_text(admin_text,tokenizer_admin,model_admin),admin_ner_dict)
    for i, ent_admin in enumerate(entidades_admin):
        entidades[indices[i]] = ent_admin
    return entidades

In [13]:
entidades = etiquetar(text)
for i, ent in enumerate(entidades):
    print("{}\t{}".format(text.split()[i],ent))

PARACETAMOL	B-ACTIVE_PRINCIPLE
500	B-FORMA_FARMA
MG	I-FORMA_FARMA
COMPRIMIDO	I-FORMA_FARMA
1	B-CANT
COMPRIMIDO	B-UND
ORAL	B-VIA_ADMIN
cada	B-PERIODICITY
6	I-PERIODICITY
horas	I-PERIODICITY
durante	B-DURATION
3	I-DURATION
dias	I-DURATION


Finalmente mostramos una manera más bonita de mostrar los datos

In [14]:
import pandas as pd

def render_pandas(ents,text_list):
    data = {'ACTIVE_PRINCIPLE':'','FORMA_FARMA':'','CANT-ADMIN':'','UND-ADMIN':'','VIA-ADMIN':'','PERIODICITY':'','DURATION':''}

    for i, ent in enumerate(ents):
        if '-ACTIVE_PRINCIPLE' in ent:
            data['ACTIVE_PRINCIPLE'] += ' ' + text_list[i]
        elif '-FORMA_FARMA' in ent:
            data['FORMA_FARMA'] += ' ' + text_list[i]
        elif '-CANT' in ent:
            data['CANT-ADMIN'] += ' ' + text_list[i]
        elif '-UND' in ent:
            data['UND-ADMIN'] += ' ' + text_list[i]
        elif '-VIA_ADMIN' in ent:
            data['VIA-ADMIN'] += ' ' + text_list[i]
        elif '-PERIODICITY' in ent:
            data['PERIODICITY'] += ' ' + text_list[i]
        elif '-DURATION' in ent:
            data['DURATION'] += ' ' + text_list[i]

    df = pd.DataFrame([data])
    return df.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).hide(axis='index')

In [15]:
render_pandas(entidades,text.split())

ACTIVE_PRINCIPLE,FORMA_FARMA,CANT-ADMIN,UND-ADMIN,VIA-ADMIN,PERIODICITY,DURATION
PARACETAMOL,500 MG COMPRIMIDO,1,COMPRIMIDO,ORAL,cada 6 horas,durante 3 dias
